In [59]:
import numpy as np 
import torch
import torch.nn as nn
import torch.nn.functional as F

### Structure

In [30]:
class GATLayer(nn.Module):
    
    """
    Basic implem
    """
    
    def __init__(self):
        
        super().__init__()
        
    def forward(self):
        

In [35]:
in_features = 5
out_features = 2
n_nodes = 3

W = nn.Parameter(torch.zeros(size=(in_features,out_features)))
nn.init.xavier_uniform_(W.data,gain=1.414)


input_f = torch.rand(n_nodes,in_features)

### Linear transformation
h = torch.mm(input_f,W)
N = h.size()[0]

print(h.shape)

torch.Size([3, 2])


In [36]:
a = nn.Parameter(torch.zeros(size=(2*out_features,1)))
nn.init.xavier_uniform_(a.data,gain=1.414)

leakyrelu = nn.LeakyReLU(0.2)  ## LeakyReLU

In [38]:
a_input = torch.cat([h.repeat(1,N).view(N*N,-1),h.repeat(N,1)],dim=1).view(N,-1,2*out_features)

In [40]:
print("a_input: ",a_input.shape)
print("a: ",a.shape)

a_input:  torch.Size([3, 3, 4])
a:  torch.Size([4, 1])


In [41]:
e = leakyrelu(torch.matmul(a_input,a))

In [45]:
e = e.squeeze(2)

In [47]:
e ## attention coefficent

tensor([[ 0.2906,  1.3567,  0.4243],
        [-0.0079,  1.0264,  0.0939],
        [-0.0379,  0.8764, -0.0112]], grad_fn=<SqueezeBackward1>)

### Masked Attention

In [49]:
adj = torch.randint(2,(3,3))

zero_vec = -9e15*torch.ones_like(e)
print(zero_vec.shape)

torch.Size([3, 3])


In [50]:
attention = torch.where(adj>0,e,zero_vec)

In [51]:
attention

tensor([[-9.0000e+15,  1.3567e+00,  4.2427e-01],
        [-7.9421e-03,  1.0264e+00, -9.0000e+15],
        [-3.7944e-02,  8.7637e-01, -9.0000e+15]], grad_fn=<SWhereBackward0>)

In [52]:
attention = F.softmax(attention,dim=1)

In [53]:
attention

tensor([[0.0000, 0.7176, 0.2824],
        [0.2622, 0.7378, 0.0000],
        [0.2861, 0.7139, 0.0000]], grad_fn=<SoftmaxBackward0>)

In [56]:
print(attention.shape)
print(h.shape)

torch.Size([3, 3])
torch.Size([3, 2])


In [57]:
h_prime = torch.matmul(attention,h)

In [58]:
h_prime

tensor([[0.5526, 0.4340],
        [0.5899, 0.2061],
        [0.5745, 0.1837]], grad_fn=<MmBackward0>)